In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt

%matplotlib inline
mpl.style.use('bmh')

In [2]:
import numpy as np

import torch
import math
import random

from tqdm import tqdm_notebook
from boltons.iterutils import pairwise
from scipy.stats import kendalltau
from itertools import permutations

from sent_order.models.kt_regression import SentenceEncoder, Regressor, Corpus, Batch
from sent_order.perms import sample_uniform_perms

In [3]:
sent_encoder = torch.load(
    '../../data/models/new/kt-reg/sent_encoder.366.bin',
    map_location={'cuda:0': 'cpu'},
)

In [4]:
regressor = torch.load(
    '../../data/models/new/kt-reg/regressor.366.bin',
    map_location={'cuda:0': 'cpu'},
)

/Users/dclure/Projects/plot-ordering/env/lib/python3.6/site-packages/torch/serialization.py:286: SourceChangeWarning: source code of class 'sent_order.models.kt_regression.Regressor' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


In [16]:
train = Corpus('../../data/dev.json/', 10000)

100%|██████████| 10000/10000 [00:02<00:00, 4197.27it/s]


In [17]:
grafs = [g for g in train.grafs if len(g.sentences) == 10]

In [18]:
len(grafs)

309

In [19]:
graf = grafs[30]

In [20]:
for s in graf.sentences:
    print(' '.join(s.tokens))
    print('\n')

This work considers coordination and bargaining between two selfish users over a Gaussian interference channel .


The usual information theoretic approach assumes full cooperation among users for codebook and rate selection .


In the scenario investigated here , each user is willing to coordinate its actions only when an incentive exists and benefits of cooperation are fairly allocated .


The users are first allowed to negotiate for the use of a simple Han - Kobayashi type scheme with fixed power split .


Conditions for which users have incentives to cooperate are identified .


Then , two different approaches are used to solve the associated bargaining problem .


First , the Nash Bargaining Solution ( NBS ) is used as a tool to get fair information rates and the operating point is obtained as a result of an optimization problem .


Next , a dynamic alternating - offer bargaining game ( AOBG ) from bargaining theory is introduced to model the bargaining process and the rates resul

In [21]:
sents = sent_encoder(graf.sentence_variables())

In [25]:
for _ in range(100):
    order = torch.randperm(len(sents))
    perm = sents[torch.LongTensor(order)]
    pred = regressor(perm.unsqueeze(0))
    print(order.tolist(), pred.data[0])

[3, 8, 4, 9, 6, 7, 1, 2, 0, 5] 0.7307759523391724
[2, 1, 3, 6, 5, 4, 7, 9, 0, 8] 0.3974292278289795
[2, 7, 9, 8, 1, 3, 4, 6, 0, 5] 0.7211130857467651
[8, 1, 0, 5, 3, 4, 7, 2, 6, 9] 0.33518990874290466
[1, 5, 2, 0, 8, 7, 6, 3, 9, 4] 0.3435349762439728
[1, 5, 0, 2, 7, 8, 3, 4, 9, 6] 0.32581430673599243
[0, 4, 6, 1, 8, 2, 7, 5, 9, 3] 0.19208624958992004
[4, 7, 1, 6, 8, 0, 2, 9, 3, 5] 0.45645007491111755
[2, 9, 6, 7, 4, 8, 3, 5, 1, 0] 0.7746144533157349
[9, 4, 1, 7, 2, 6, 3, 0, 8, 5] 0.6494330763816833
[0, 9, 6, 4, 7, 1, 8, 2, 5, 3] 0.3791032135486603
[0, 1, 5, 8, 3, 4, 9, 7, 6, 2] 0.2830505967140198
[8, 9, 5, 1, 0, 6, 3, 2, 7, 4] 0.6298239231109619
[6, 3, 9, 5, 7, 4, 2, 1, 8, 0] 0.7352059483528137
[1, 9, 7, 4, 5, 0, 6, 3, 8, 2] 0.4884956479072571
[4, 6, 9, 8, 0, 2, 7, 3, 1, 5] 0.5243723392486572
[8, 0, 1, 2, 6, 3, 9, 4, 7, 5] 0.2986418902873993
[5, 4, 9, 0, 8, 3, 2, 1, 6, 7] 0.5152631402015686
[3, 4, 5, 9, 7, 2, 6, 8, 0, 1] 0.8178740739822388
[3, 8, 0, 5, 1, 9, 6, 4, 7, 2] 0.4307526051998

In [22]:
def predict(graf):
    
    sents = sent_encoder(graf.sentence_variables())
    
    perms = list(permutations(range(len(sents))))
    
    x = torch.stack([
        sents[torch.LongTensor(perm)]
        for perm in perms
    ])
    
    scores = regressor(x)
    
    return perms[torch.min(scores, 0)[1].data[0]]

In [ ]:
kts = []
for g in tqdm_notebook(grafs):
    pred = predict(g)
    kt = kendalltau(pred, range(len(pred))).correlation
    kts.append(kt)

HBox(children=(IntProgress(value=0, max=1667), HTML(value='')))

In [18]:
np.mean(kts)

0.86951693503609107

In [19]:
kts.count(1) / len(kts)

0.6879511382565242